In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace

In [3]:
import ecephys_analyses as ea

In [20]:
ea.get_channels("Valentino", "sevo-dex", "imec0", "all")[193:308]

array([  2,   5,   6,   9,  10,  13,  14,  17,  18,  21,  22,  25,  26,
        29,  30,  33,  34,  37,  38,  41,  42,  45,  46,  49,  50,  53,
        54,  57,  58,  61,  62,  65,  66,  69,  70,  73,  74,  77,  78,
        81,  82,  85,  86,  89,  90,  93,  94,  97,  98, 101, 102, 105,
       106, 109, 110, 113, 114, 117, 118, 121, 122, 125, 126, 129, 130,
       133, 134, 137, 138, 141, 142, 145, 146, 149, 150, 153, 154, 157,
       158, 161, 162, 165, 166, 169, 170, 173, 174, 177, 178, 181, 182,
       185, 186, 189, 190, 193, 194, 197, 198, 201, 202, 205, 206, 209,
       210, 213, 214, 217, 218, 221, 222, 225, 226, 229, 230])